# Análisis y Proceso de los datos provenientes del PDM

Este `jupyter` y los relacionados contiene el análisis y el proceso que permite la homologación/conversión entre los datos en formato `.csv` provenientes de la base de datos de los Planes de Desarrollo Municipal (PDM) de Honduras y el modelo de datos de Fonsagua.

En concreto en este `jupyter` se analizan la equivalencia entre campos entre los datos del PDM y los de Fonsgua y se realizan los procesos para poder realizar la actualización de los datos de comunidades a partir de los datos del PDM o incorporar nuevas comunidades.

Como origen de datos se usarán los datos ya procesado, `pdm.xlsx`, pestaña `Datos_PDM`

In [1]:
# Configuración básica de jupyter, pandas, ...
# e importación de librerías que necesitaremos durante el análisis

# https://medium.com/@1522933668924/using-matplotlib-in-jupyter-notebooks-comparing-methods-and-some-tips-python-c38e85b40ba1
# Para hacer los mapas interactivos

# %matplotlib widget 
%matplotlib inline 
import numpy as np
from IPython.display import display, HTML, Markdown
# Ejemplos de uso
# display(HTML('<h2 style="color:red;">Subtitulo "YYY - NOMBRE"</h2>'))
# display(Markdown("# Título principal"))
# display(HTML(df.describe().to_html()))  # muestra todos los datos
import pandas
from ietl import pandas_utils

from ietl.geopandas_utils import (
    get_dataframe_from_spatialite_table,
    plot_polygon_labels,
)
import geopandas as gpd

pandas.options.mode.use_inf_as_na = True
# http://songhuiming.github.io/pages/2017/04/02/jupyter-and-pandas-display/
# pandas.set_option('display.max_rows', 500)
# pandas.set_option('display.max_columns', 500)


# https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html
# Si se está trabajando con módulos externos usar %autoreload 2 evita tener que
# recargar a mano. %autoreload 0 lo desactiva
%load_ext autoreload
%autoreload 2



# Ejecuta el .ipyndb referenciando como si fuera en este propio Lab
# Todas las variables del otro pasan a estar disponible en este
# Cualquier salida por pantalla del otro se muestra en los resultados 
# de la celda donde se ejecuta el %run
# if __name__ == '__main__' and '__file__' not in globals():
#     print("Este código sólo se ejecuta cuando lo llamo directamente y no a través de un `%run`")
# %run nasmar_cartografia.ipynb

In [2]:
from nasmar import common_left_field, common_right_field
import nasmar

sqlite_path = "191208_fonsagua_todos_municipios.sqlite"
excel_datos_pdm_path = "pdm.xlsx"

# Sobre el modelo de datos, la homologación y los datos proporcionados

Los csv proporcionados sólo contienen los datos correspondientes a la tabla `comunidades` del modelo de datos de Fonsagua. Durante la fase de análisis se identificaron más tablas de Fonsagua que podían ser rellenadas a partir de la información de los PDM:

* entrevistadores
* entrevistados
* tipo_cultivos
* ganaderia

Ninguno de estos datos ha sido proporcionado.

# Otros comentarios sobre los datos proporcionados

* Los nombres de campos de los csv proporcionados no coinciden (en los casos en que son iguales) con los del modelo de datos de Fonsagua. Esto hace que haya que establecer un mapeo uno a uno de los campos. No es un problema, pero es interesante pensar si a futuro tiene sentido estandarizar estos nombres entre ambos modelos.

**REVISAR ICARTO**

Para los campos que listo a continuación, hay información asociada en el mdd, pero en la parte de "Está disponible" no aparece un "Si". ¿Qué significa esto? ¿Debería habernos proporcionado los datos o no?

* cp_otros
* f_indus_for
* f_indus_inf
* f_comer_for
* f_comer_inf
* prop_area_cultivada
* arre_area_cultivada
* f_medias
* med_area_cultivada
* produccion


Comentarios genéricos sobre como está hecho la homologación.

* Tratar de no usar nombres duplicados para el header. "DOMINIO" aparece dos veces
* Tratar de minimizar la importancia de los colores. A la hora de que otra persona coja el excel es difícil que sepa que significa un tipo de letra rojo o un fondo amarillo. Además es muy difícil de parsear para realizar automatismos. Por ejemplo se puede añadir una columna que indique que la información es dudosa, o en este caso en el "Está disponible" igual se puede poner "Dudoso".

In [2]:
departamentos = nasmar.get_departamentos_bd()
municipios = nasmar.get_municipios_bd()
caserios = nasmar.get_caserios_nasmar()
# comunidades = nasmar.get_comunidades_bd()

pdm = pandas.read_excel(excel_datos_pdm_path, sheet_name="Datos_PDM", dtype = {"codigo_comunidad": object, "sort_code": object})
pdm.set_index("codigo_comunidad", drop=True, inplace=True, verify_integrity=True)  # En dos pasos para poder setearlo como object

# display(pdm.columns.to_list())
pdm

,nombre_comunidad,nombre_municipio,total_viviendas,total_familias,total_personasv,total_sectorpri,total_sectorsec,total_sectorter,total_sectorcom,total_sectorind,...,total_hombres5a18,total_mujeres18a60,total_hombres18a60,total_adultos,total_ancianosm60,total_ancianasm60,total_ancianos,meta_nombre_comunidad_mal,nombre_comunidad_sin_codigo,sort_code
codigo_comunidad,,,,,,,,,,,,,,,,,,,,,
060601001001,001 - BO. EL CENTRO,EL TRIUNFO,17,22,69,0,0,0,0,0,...,13,16,21,39,2,1,3,False,BO. EL CENTRO,060601001
060601001002,002 - COL. BELLA HORIZONTE,EL TRIUNFO,71,77,301,4,0,0,2,0,...,43,67,88,165,10,8,18,False,COL. BELLA HORIZONTE,060601001
060601001003,003 - BO. EL TOLOLO,EL TRIUNFO,149,156,605,9,32,26,19,3,...,76,135,172,325,28,34,62,False,BO. EL TOLOLO,060601001
060601001004,004 - BO. BRISAS DEL RIO GUALE,EL TRIUNFO,71,88,289,17,2,1,4,1,...,45,57,87,154,11,19,30,False,BO. BRISAS DEL RIO GUALE,060601001
060601001005,005 - BO. EL CALVARIO,EL TRIUNFO,44,51,209,3,3,1,20,0,...,39,43,55,108,7,6,13,False,BO. EL CALVARIO,060601001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170910005000,005 - LAS LOMAS,SAN LORENZO,3,3,9,0,0,0,0,0,...,0,3,4,7,0,0,0,False,LAS LOMAS,170910005
170910006000,006 - LAS PILAS,SAN LORENZO,28,29,100,2,0,0,0,0,...,7,26,30,57,3,2,5,False,LAS PILAS,170910006
170910007000,007 - VOLCANCITO,SAN LORENZO,26,30,94,0,0,0,0,0,...,7,26,26,54,5,3,8,False,VOLCANCITO,170910007


In [3]:
# Cargamos el excel con el modelo de datos de la homologación
mdd_path = "190717_MDD_comunidades.xls"
mdd_xls_options = {
    "sheet_name": "comunidades",
    "header": 0,
    # "names": ["NOMBRE BDD"],
    "index_col": "NOMBRE BDD",
    # "usecols": No podemos usarlo peta al haber nombres duplicados
}
mdd = pandas.read_excel(mdd_path, **mdd_xls_options)

# Elinamos las columnas que tienen menor interés
# usecols = ["NOMBRE BDD", "NOMBRE", "TIPO DE CAMPO I (texto, entero, real, fecha, booleano", "TIPO DE CAMPO II (CB, RB, ChB)", "DOMINIO", "COMPROBACIONES O RELACIONES", "EDITABLE", "OBLIGATORIO", "COMENTARIOS", "Esta disponible", "Pantalla", "Nombre campo BDD", "Conversión", "TIPO(tamaño)", "DOMINIO", "Comentario", "Comentario iCarto" ],
# NOMBRE_BDD
usecols = ["NOMBRE", "TIPO DE CAMPO I (texto, entero, real, fecha, booleano", "TIPO DE CAMPO II (CB, RB, ChB)", "DOMINIO", "COMPROBACIONES O RELACIONES", "EDITABLE", "OBLIGATORIO", "COMENTARIOS", "Esta disponible", "Pantalla", "Nombre campo BDD", "Conversión", "TIPO(tamaño)", "DOMINIO.1", "Comentario", "Comentario iCarto"]
# nombre
renamecols = ["nombre_largo", "tipo", "tipo_widget", "dominio", "comprobaciones", "editable", "obligatorio", "comentario", "disponible", "pdm_pantalla", "pdm_nombre", "pdm_conversión", "pdm_tipo", "pdm_dominio", "pdm_coment", "pdm_coment_icarto"]
mdd = mdd[usecols]
mdd.set_axis(renamecols, axis='columns', inplace=True)

# Marcamos algunas filas adicionales con un "Dudoso"
# Son homologable o al menos dudosos los que tienen algún valor en "Esta disponible" o "Pantalla"
# Tenemos algunos campos en rojo que no se lo que significan
# Y eliminamos todos los campos para los que estamos seguros de que no hay homologación
mdd.loc[mdd["pdm_pantalla"].notna() & mdd["disponible"].isna(), "disponible"] = "Dudoso"
homologables = mdd[mdd.disponible.notna()]
homologables = homologables.drop("pdm_pantalla", axis="columns")

print(homologables.loc["f_primario", ["comprobaciones", "tipo", "nombre_largo", "editable", "pdm_conversion"]])

comprobaciones    Automático: (f_c_propia +f_c_ajena)/n_familias...
tipo                                                           real
nombre_largo                                   Sector primario (%):
editable                                                        NaN
pdm_conversion                                                  NaN
Name: f_primario, dtype: object


/var/development/virtualenvs/fonsagua/lib/python3.6/site-packages/pandas/core/indexing.py:961: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return getattr(section, self.name)[new_key]


In [9]:
# mapeo_mdd_csv = {f:None for f in homologables.index} # Se puede usar para hacer un diccionario inicial que luego se rellena a mano

def mypop(l):
    """
    returns the first element of the list if exists or none
    """
    # return next(iter(l or []), None)
    return next(iter(l if (l.size > 0) else []), None)

def tareas_a_manzanas(t):
    # 1 manzana = 16 tareas
    m = t / 16
    return m

mapeo_mdd_csv = {
 'comunidad': "nombre_comunidad_sin_codigo", # Hay que eliminar los primeros caracteres que son el código
 'chequeo_comunidad_ine': lambda row: mypop(caserios[caserios["codigo"] == row.name[:9]]["nombre"].values),  # incluyo el nombre del caserio del ine para chequeos
 'cod_comunidad': "codigo_comunidad",  # es el index
 'departamento': lambda row: departamentos[departamentos["codigo"] == row.name[:2]]["nombre"].values[0],
 'cod_departamento': lambda row: row.name[:2],
 'municipio': lambda row: municipios[municipios["codigo"] == row.name[:4]]["nombre"].values[0],  # Se prefiere usar el de la cartografía base y no el del municipio que está todo en mayúsculas 
 'cod_municipio': lambda row: row.name[:4], # Está en los datos originales pero es más lógico generarlo
 'n_familias': "total_familias", # revisar
 'n_viviendas': "total_viviendas", # revisar
 'n_habitantes': "total_personasv", # revisar ¿en nuestros propios datos deste número debería ser la suma de resto de d_ninhos, ... o no? en los datos que nos pasan coincide?. Mejorar en el modelo. Igual un campo automático o clarificar como debería ser
 'n_ninhas': "total_ninasm5",
 'n_ninhos': "total_ninosm5",
 'tot_ninhos': "total_ninos", # automatico n_ninhas + n_ninhos - verificar suma
 'n_muj_jovenes': "total_mujeres5a18",
 'n_hom_jovenes': "total_hombres5a18",
 'n_mujeres': "total_mujeres18a60",
 'n_hombres': "total_hombres18a60",
 'tot_adultos': "total_adultos", # revisar, en nuestro mdd es de 5 a 60 (mejorar campo nombre). Mientras que total_adultos es de 18 a 60. automático.
 'n_ancianas': "total_ancianasm60",
 'n_ancianos': "total_ancianosm60",
 'tot_ancianos': "total_ancianos", # automatico n_ancianas + n_ancianos - verificar suma
    
 # En PDM los datos de f_primario, f_secundario, f_terciario sólo se cogen para la persona 
 # que realiza la encuesta. Por tanto a pesar de que se dan como número de personas, se
 # se podrían asimilar a número de familias que es lo que se recoge en Fonsagua
    
 # Toda esta parte de a que se dedica la gente si primario, si otra cosa es bastante confusa
 # y me da que hay un problema gordo con los PDM. En la boleta pone:
 # Si contesto que su ocupación actual es como empleado, en que sector esta contratado: Comercial, Industria, Servicio
 # Si contesto que su ocupación actual es por cuenta propia, en que sector esta contratado: Primario, Secundario, Terciario
 # Pero nosotros para primario ya distinguimos entre cuenta propia y ajena (empleado) así que el resultado quedaría muy raro.
 # En la homologación se proporciona como podría ser, pero aún así los resultados podrían resultar poco fiables.

 # 'f_primario': None, # Es un campo autocalculado, no podemos insertar directamente en él
 # f_primario = (f_c_propia + f_c_ajena)/n_familias)*100
 # f_c_propia y f_c_ajena también son autocalculados   
 # f_c_propia = cp_granos + cp_ganaderia + cp_frutales + cp_otros 
 # f_c_ajena = ca_cafe + ca_canha + ca_frutales + ca_otros

 # PMD sólo recoge café, .. no frutales y cosas asi, por lo que es difícil hacer sumas y cosas con sentido
 'cp_otros': 'total_sectorpri', # Total personas que trabajan en el sector primario (cuenta propia), al meterlo aquí suma bien para f_primario
 
 # 'f_secundario': None, # Es un campo autocalculado, no podemos insertar directamente en él
 # f_secundario = (f_industria+f_construccion+f_maquila+f_otros_sec)/n_familias )*100
 # Asimilamos formal a cuenta propia e informal a cuenta ajena
 # 'f_industria': None, # Es autocalculado
 # 'f_construccion': None, # Es autocalculado
 'f_indus_for': 'total_sectorind',
 'f_indus_inf': 'total_sectorsec',
 
    
 # 'f_terciario': None, # Es un campo autocalculado, no podemos insertar directamente en él
 # f_terciario = (f_comercio+f_otros_ter)/n_familias )*100
 # 'f_comercio' = None,  # Es autocalculado. f_comer_for + f_comer_inf
 # Pasa como en industria lo de formal/informal y confusión respecto a "Comercial" y "Servicios" de la boleta
 'f_comer_for': lambda row: row['total_sectorcom'] + row['total_sectorser'],
 'f_comer_inf': 'total_sectorter',
 

 # Sin los datos en bruto es difícil saber que estamos metiendo aquí, si son sólo las que la tienen
 # pagado o no, ... no sabemos que hacer con los datos de comunal (aunque esto puede ser el "a medias", ...
 # No nos pasó los datos desagregados por sexo
 'f_propietarias': 'total_tenenciapropia',
 'prop_registrada': None,
 'prop_tramites': None,
 'prop_area_cultivada': None,
 'f_arrendatarias': 'total_tenenciaalquilada',
 'f_medias': None,

 # Sólo nos pasó el dato de `'total_areatareas', Area de siembro en tareas`. No lo tenemos desagregado
 # por tipo de tenencia que es como tenemos el dato en Fonsagua
 'arre_area_cultivada': None,
 'med_area_cultivada': None,
 'area_cultivada': None,

 'tip_cultivos': None,
 'produccion': 'total_ganosb', # produccion es quintales/familia/año. Nos fiamos de que 'total_ganosb' use esa misma medida
 'sup_total_riego': lambda row: tareas_a_manzanas(row['total_riegotareas']),
 'n_vacas': 'total_bovino',
 'n_gallinas': 'total_aves',
 'n_cerdos': 'total_porcino',
 'n_burros': 'total_equino',
 
 # Nosotros medimos familias propietarias o no de la vivienda. Y sólo consideramos la tenencia si está totalmente pagada
 # En los PDM tienen una casilla concreta cuando todavía la están pagando. Y se puedo marcar hombre, mujer o ambos en quien la posee por 
 # tanto pueden salir resultados muy raros sin partir de los datos en bruto
 # Estos resultados no van a tenir ningún sentido
 # Siempre da negativo. Es decir que seguramente están midiendo personas y no familias o algo así
 'chequeo_tenenencia_familias': lambda row: row["total_tenenciahom"] + row["total_tenenciamuj"] + row["total_notenencia"] - row["total_familias"],
 # Esto da mucho más negativos, así que no miden personas exactamente
 'chequeo_tenenencia_personas': lambda row: row["total_tenenciahom"] + row["total_tenenciamuj"] + row["total_notenencia"] - row["total_personasv"],
 # Hay bastantes 0 pero también bastantes negativos
 'chequeo_tenenencia_viviendas': lambda row: row["total_tenenciahom"] + row["total_tenenciamuj"] + row["total_notenencia"] - row["total_viviendas"],
 'f_viv_prop': lambda row: row["total_tenenciahom"] + row["total_tenenciamuj"], # No se puede hacer así porqué va a sumar también en el caso de ambos
 'viv_prop_hombres': lambda row: row["total_tenenciahom"],
 'viv_prop_mujeres': lambda row: row["total_tenenciamuj"],
 'f_viv_noprop': lambda row: row["total_notenencia"], # Y eso no está claro sin son personas, familias, ...
    
 # Miden los materiale de construccion de forma un poco distinta.
 'viv_bahareque': 'total_matbahareque',
 'viv_adobe': 'total_matabobe',
  # Debería haberlo proporcionado como la suma de los restantes tipos. Se podría calcular como viviendas - bahareque - adobe. Pero nuestro "mixto" es confuso hay muchas comunidades que lo tienen
 'viv_otros': lambda row: row["total_viviendas"] - row["total_matbahareque"] - row["total_matabobe"], 
    
 # Nos pasó datos de cocina de leña, Ecofogón y Electricidad. Pero hay más valores posibles   
 'coc_lenha': "total_cocinalena",
 'coc_lenha_mej': lambda row: row['total_cocinaeco'], # se le podría sumar row['total_cocinaelec'] pero no se si tiene sentido
 # Esto nos dice que leña + ecofogón + electricidad es en general distinto al número de viviendas, por lo que seguramente no tiene sentido sumar la electricidad
 # En los datos de fonsagua 'coc_lenha' + 'coc_lenha_mej' casi siempre es igual al número de viviendas así que no tengo claro lo que hacer
 'chequeo_cocina': lambda row: row['total_cocinaeco'] + row['total_cocinaelec'] + row['total_cocinalena'] - row['total_viviendas'],
    
 'electricidad': lambda row: True if row["total_vivconele"] else False, # Si hay alguna con electricidad es que hay electricidad
 'viv_electricidad': "total_vivconele",
 # Con 75% son 488 sin y 114 con (81% de las viv sin). Con 50% son 332 sin y 270 con.
 # Se pone al 75% encaja más con el dato actual (74% de las comunidades sin 28 de 107 tienen.)
 'alumbrado': lambda row: row['total_vivconapub'] > row["total_viviendas"] * 0.75,
    
 'telf_fijo': lambda row: row['total_vivcontelf'] > row['total_viviendas'] * 0.5,
 'telf_movil': lambda row: row['total_vivcontelm'] > row['total_viviendas'] * 0.5,
    
 'muertes_ninhos': None,   
 'enf_tip_adultos': None,
 
    
 # Faltan muchos datos que nos podría proporcionar
 # 'sist_abastecimiento': None, # Si / No / Parcial
 'n_viv_abast': None, # cuando sist_abastecimiento es parcial
 'aba_s_cantareras': None,
 'aba_s_domiciliar': 'total_vivconaguaa', # Total de vivienda que cuentan con agua de acueducto
 'aba_s_nacimiento': None,
 'aba_s_rio': 'total_vivconaguar', # Total de vivienda que cuentan con agua de rio
 'aba_s_lluvia': 'total_vivconaguall',  # Total de vivienda que cuentan con agua lluvia
 'aba_s_broquel': None,
 'aba_s_broquel_com': None,
 'tot_s_sin_abast': None,
    
 # en general esta suma no da por lo que no tengo ni idea de como se metieron estos datos o que nos está pasando  
 'chequeo_letrinas': lambda row: row['total_vivconfosasi'] + row['total_vivconfosase'] - row['total_vivconlet'],
 'let_vivienda': lambda row: row['total_vivconfosasi'] + row['total_vivconfosase'],
 'let_hoyo': 'total_vivconfosasi',
 'let_septica': 'total_vivconfosase',
 # se ha puesto como el 50%, pero en fonsagua este dato está confuso. Está en blanco en muchos sitios, hay puestos verdaderos con mucho menos
 # del 50%, ...
 'uso_letrinas': lambda row: row['total_vivconlet'] > (row['total_vivconfosasi'] + row['total_vivconfosase']) * 0.5,
 
 'disp_basuras': None,
 'geom': None,
}

nombres_campos_homologables = {k for k in (homologables.index.to_list() + ['geom']) if not k.startswith("chequeo_")}
nombres_campos_mapeo_mdd_csv = {k for k in mapeo_mdd_csv.keys() if not k.startswith("chequeo_")}
# display(nombres_campos_homologables)
if nombres_campos_homologables != nombres_campos_mapeo_mdd_csv:
    display(nombres_campos_homologables.symmetric_difference(nombres_campos_mapeo_mdd_csv))
    # raise Exception("Los campos homologables son distintos a lo esperado")


{'cp_granos',
 'f_comercio',
 'f_primario',
 'f_secundario',
 'f_terciario',
 'telf_movil'}

In [10]:
pdm_ajustado = pandas.DataFrame(index=pdm.index.copy())


pdm_ajustado["comunidad"] = pdm.loc[:, "nombre_comunidad"]

# https://stackoverflow.com/questions/26658240/
def nombre_departamento(row):
    a = departamentos[departamentos["codigo"] == row.name[:2]]["nombre"].values[0]
    return a


for k, v in mapeo_mdd_csv.items():
    if k == "cod_comunidad":
        continue
    if callable(v):
        pdm_ajustado[k] = pdm.apply(v, axis='columns')
    elif v:
        pdm_ajustado[k] = pdm.loc[:, v]
    else:
        print("Revisar: ", k, v)
        

# pdm_ajustado["telf_fijo"].value_counts()
# pdm_ajustado["chequeo_telf_fijo_max_de_movil_o_fijo"].value_counts()
# pdm_ajustado["alumbrado"].value_counts()
# pdm_ajustado["chequeo_tenenencia_viviendas"].values
# pdm_ajustado["chequeo_tenenencia_personas"].values
# pdm_ajustado["chequeo_tenenencia_viviendas"].values
# pdm_ajustado["chequeo_letrinas"].values
pdm_ajustado.to_excel("pdm_ajustado_a_fonsagua.xlsx")


Revisar:  prop_registrada None
Revisar:  prop_tramites None
Revisar:  prop_area_cultivada None
Revisar:  f_medias None
Revisar:  arre_area_cultivada None
Revisar:  med_area_cultivada None
Revisar:  area_cultivada None
Revisar:  tip_cultivos None
Revisar:  muertes_ninhos None
Revisar:  enf_tip_adultos None
Revisar:  n_viv_abast None
Revisar:  aba_s_cantareras None
Revisar:  aba_s_nacimiento None
Revisar:  aba_s_broquel None
Revisar:  aba_s_broquel_com None
Revisar:  tot_s_sin_abast None
Revisar:  disp_basuras None
Revisar:  geom None


In [13]:
pdm_ajustado["sort_code"] = pdm_ajustado.index.str[:9]

# Leemos la tabla de comunidades de la última base de datos
# y generamos un DataFrame con las columnas homologables
# Si hay algún código fuera de un municipio NASMAR lanzamos un error
comunidades = nasmar.get_comunidades_bd()

# Contamos el número de elementos del df que tenemos por municipio
# display(nasmar.number_of_elements_by_council(comunidades, "cod_comunidad"))

# comunidades.set_index("cod_comunidad")
campos_comunidades_merge = [k for k, v in mapeo_mdd_csv.items() if not k.startswith("chequeo") and v]
comunidades_merge = comunidades[campos_comunidades_merge]
# display(comunidades_merge)

comunidades_pdm_ajustado_merge_options = {
    "left_on": "cod_comunidad",
    "right_on": "sort_code",
    "how": "inner",
    "indicator": False,
    "validate": "one_to_many",
    "suffixes": ("_fonsagua", "_pdm"),
}

enlace_comunidades_pdm_ajustado = pandas.merge(comunidades_merge, pdm_ajustado, **comunidades_pdm_ajustado_merge_options)
enlace_comunidades_pdm_ajustado.set_index("cod_comunidad", inplace=True)
enlace_comunidades_pdm_ajustado = enlace_comunidades_pdm_ajustado.reindex(sorted(enlace_comunidades_pdm_ajustado.columns), axis=1)
enlace_comunidades_pdm_ajustado.drop(columns=["departamento_fonsagua", "cod_departamento_fonsagua", "cod_municipio_fonsagua", "sort_code", "cod_departamento_pdm", "cod_municipio_pdm", "departamento_pdm", "municipio_pdm"], inplace=True)
enlace_comunidades_pdm_ajustado_minimo = enlace_comunidades_pdm_ajustado[["chequeo_comunidad_ine", "comunidad_fonsagua", "comunidad_pdm", "n_familias_fonsagua", "n_familias_pdm", "n_habitantes_fonsagua", "n_habitantes_pdm", "n_viviendas_fonsagua", "n_viviendas_pdm"]]
with pandas.ExcelWriter("enlace_comunidades_pdm_ajustado.xlsx") as writer:
    enlace_comunidades_pdm_ajustado.to_excel(writer, sheet_name='todos_campos')
    enlace_comunidades_pdm_ajustado_minimo.to_excel(writer, sheet_name='campos_basicos')

# enlace_comunidades_pdm_ajustado